In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from downhole.survey import Survey, DownholeSurvey

from downhole.examples.ex_survey import *

s = Survey()

import pandas as pd

In [3]:
survey_df = pd.read_excel('.\\Test Spreadsheets\\Survey.xlsx', index_col=0)
collar_df = pd.read_excel('.\\Test Spreadsheets\\Collar.xlsx', index_col=0)

In [4]:
holes = list(pd.unique(collar_df.index))
for hole in holes:
    try:
        if len(survey_df.loc[[hole]]) == 1:
            print(f"No Downhole Survey {hole}, collar info used.")
            ds = DownholeSurvey(
                [collar_df.at[hole, 'Azimuth_GridN'],collar_df.at[hole, 'Azimuth_GridN']],
                [collar_df.at[hole, 'DIP'],collar_df.at[hole, 'DIP']],
                [0,collar_df.at[hole, 'TotalDepth_m']],
                collar_df.at[hole, 'UTM_N83_EASTING'],
                collar_df.at[hole, 'UTM_N83_NORTHING'],
                collar_df.at[hole, 'UTM_N83_ELEV_MSL'], hole
            )
        else:
            azm_data = list(survey_df.loc[[hole]]['AzimGridN'])
            dip_data = list(survey_df.loc[[hole]]['Final_Dip'])
            depth_data = list(survey_df.loc[[hole]]['Depth_m'])
            
            if collar_df.at[hole, 'TotalDepth_m'] > (depth_data[-1] + 25):
                difference = collar_df.at[hole, 'TotalDepth_m'] - depth_data[-1]
                print(f"Collar depth > survey depth by 25m for {hole}, interval added. {difference}")
                azm_data.insert(len(azm_data),azm_data[-1])
                dip_data.insert(len(dip_data),dip_data[-1])
                depth_data.insert(len(depth_data),collar_df.at[hole, 'TotalDepth_m'])
            
            
            if min(survey_df.loc[[hole]]['Depth_m']) > 9:
                print(min(survey_df.loc[[hole]]['Depth_m']),azm_data[0], dip_data[0])
                print(collar_df.at[hole, 'Azimuth_GridN'], collar_df.at[hole, 'DIP'])
                print(f"No Depth=0.00 reading for {hole}, interval added.")
                azm_data.insert(0,azm_data[0])
                dip_data.insert(0,dip_data[0])
                depth_data.insert(0,0.00)
            
            ds = DownholeSurvey(
                azm_data,
                dip_data,
                depth_data,
                collar_df.at[hole, 'UTM_N83_EASTING'],
                collar_df.at[hole, 'UTM_N83_NORTHING'],
                collar_df.at[hole, 'UTM_N83_ELEV_MSL'], hole
            )
        

    except KeyError:
        print(f"No Downhole Survey or record of {hole}, collar info used.")
        ds = DownholeSurvey(
            [collar_df.at[hole, 'Azimuth_GridN'],collar_df.at[hole, 'Azimuth_GridN']],
            [collar_df.at[hole, 'DIP'],collar_df.at[hole, 'DIP']],
            [0,collar_df.at[hole, 'TotalDepth_m']],
            collar_df.at[hole, 'UTM_N83_EASTING'],
            collar_df.at[hole, 'UTM_N83_NORTHING'],
            collar_df.at[hole, 'UTM_N83_ELEV_MSL'], hole
        )
    
    s.add_downhole_survey(ds)


No Downhole Survey KDI-12-001, collar info used.
No Downhole Survey KDI-12-002, collar info used.
11.0 86.2 -51.07
86.3 -50.0
No Depth=0.00 reading for KDI-13-018, interval added.
No Downhole Survey KDI-13-019, collar info used.
No Downhole Survey KDI-13-023, collar info used.
No Downhole Survey KDI-13-028, collar info used.
No Downhole Survey KDI-13-031, collar info used.
Collar depth > survey depth by 25m for KDI-14-065, interval added. 42.72000000000003
Collar depth > survey depth by 25m for KDI-15-001, interval added. 277.62
No Downhole Survey KDI-15-006b, collar info used.
No Downhole Survey KDI-15-010a, collar info used.
No Downhole Survey KDI-15-025b, collar info used.
No Downhole Survey KDI-15-035a, collar info used.
No Downhole Survey KDI-15-040, collar info used.
Collar depth > survey depth by 25m for KDI-15-048b, interval added. 27.0
Collar depth > survey depth by 25m for KDI-15-051a, interval added. 46.0
Collar depth > survey depth by 25m for KDI-16-052, interval added. 26.

In [ ]:
s.export_shapefile()

In [7]:
s.downhole_surveys[15].min_curv.sample_point(55)

TypeError: '<' not supported between instances of 'list' and 'int'

In [ ]:
s.downhole_surveys[15].min_curv.sample_point(55)